In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
# Read review json in chunks
review_itr = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json",
                          lines=True,
                          orient='columns',
                          chunksize=1000000)

# Load user json
user_itr = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json",
                           lines=True,
                           orient='columns',
                          chunksize=1000000)

In [4]:
# consider only first chunk of size 1M and break

for review in review_itr:
    review_df = review
    break
    
for user in user_itr:
    user_df = user
    break

In [5]:
# only consider reviews having 3, 4 and 5 stars given by users

review_df = review_df[review_df.stars.isin([3,4,5])]
review_df.shape

In [6]:
# only consider reviews which were useful for atleast 1 person

# review_df = review_df[review_df.useful!=0]
# review_df.shape

In [7]:
# extract year from review date

review_df['date'] = pd.to_datetime(review_df['date'],errors='coerce')
review_df['Year'] = review_df.date.dt.strftime('%Y')

# only consider reviews from past 5 years

review_df = review_df[review_df.Year.isin(['2017','2018','2019','2020','2021'])]
review_df.shape

In [8]:
# Load business json
business_df = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json",
                           lines=True,
                           orient='columns')

In [9]:
# # Load tip json
# tip_df = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json",
#                            lines=True,
#                            orient='columns')

In [9]:
print(business_df.shape)
business_df.head(2)

In [10]:
# consider only open businesses and are restaurants

business_df = business_df[(business_df.is_open==1) &
                          (business_df.categories.str.contains('Restaurants*'))
                         ]

business_df.shape

In [11]:
# Remove businesses where attributes column in None

business_df = business_df[~business_df.attributes.isna()]
business_df.shape

In [12]:
business_df.head(2)

In [13]:
# consider only restaurants which provide table and take out service
business_df = business_df[business_df.attributes.apply(lambda x:x.get('RestaurantsTableService','RestaurantsTakeOut')=='True')]
business_df.shape

In [14]:
# targeting restaurants having review count above certain limit
business_df = business_df[(business_df.review_count>=5)]#.shape
# & (business_df.review_count<=1000)].shape
business_df.shape

In [15]:
# check distribution of states in our data
business_df.state.value_counts()

In [16]:
# considering only 1 state - Massachusetts for this task

# business_df = business_df[business_df.state == 'MA']
# business_df.shape

In [17]:
#choose businesses whose review count is more than a certain limit in review_df
review_df = review_df[(review_df['business_id'].isin(business_df.business_id.values)) &
         (review_df.groupby('business_id')['business_id'].transform('size')>=7)
         ]#.shape
review_df.shape

In [18]:
#choose cities where restaurants are more than a certain limit

business_df = business_df[(business_df.business_id.isin(review_df.business_id.unique())) &
           (business_df.groupby('city')['city'].transform('size')>=5)]

business_df.shape

In [19]:
# keep reviews only for our final curated list of restaurants in review df

review_df = review_df[review_df.business_id.isin(business_df.business_id.values)]
review_df.shape

In [20]:
review_df.head(2)

In [21]:
# add city and business name column to review df

review_df = review_df.merge(business_df[['business_id','city','name']], how='left', on='business_id')

In [22]:
review_df.head(2)

### building user-review base

In [23]:
for review in review_itr:
    review_df2 = review
    break

In [24]:
# only consider reviews having 4 and 5 stars given by users

review_df2 = review_df2[review_df2.stars.isin([4,5])]
review_df2.shape

In [25]:
# only consider reviews which were useful for atleast 1 person

review_df2 = review_df2[review_df2.useful!=0]
review_df2.shape

In [26]:
# extract year from review date

review_df2['date'] = pd.to_datetime(review_df2['date'],errors='coerce')
review_df2['Year'] = review_df2.date.dt.strftime('%Y')


# only consider reviews from past 5 years

review_df2 = review_df2[review_df2.Year.isin(['2017','2018','2019','2020','2021'])]
review_df2.shape

In [27]:
# consider only users which have reviews in our review_df2 chunk

user_df = user_df[user_df.user_id.isin(review_df2.user_id.unique())]
user_df.shape

In [28]:
user_df.head(2)

In [29]:
# consider only users whose counts of fans and usefulness of reviews are beyond certain limit

user_df = user_df[(user_df.fans>200) & (user_df.useful>2000)]
user_df.shape

In [28]:
user_df.columns

In [30]:
# consider only those reviews whose users are present in user_df

review_df2 = review_df2[review_df2.user_id.isin(user_df.user_id.unique())]
review_df2.shape

In [73]:
review_df2.head(2)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

In [37]:
# initialize tfidf vectorizer to vectorize the reviews
max_len = 300
tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 3), # also consider bigram, trigram
                     min_df=1, # atleast one occurence of word
                     max_features=max_len, # num of features i.e. embedding dims
                     stop_words='english' #remove frequent unnecessary words
                    ) 

In [68]:
# function to fetch user reviews from review df2 sorted by stars given, usefulness and recency

def get_top_user_reviews(u_id):
    user_vec =  review_df2[review_df2.user_id==u_id].sort_values(['stars','useful','Year'],
                                                 ascending=[False,False,False]
                                                )['text'].head(10).values
#     print(user_vec[:5])
    
    user_vec = tf.fit_transform([' '.join(user_vec)]).toarray()
    print(user_vec.shape)
    if user_vec.shape[1]<max_len:
        user_vec = np.pad(user_vec,pad_width = [(0,0),(0,max_len-user_vec.shape[1])])
    
    return user_vec

In [50]:
# take sample user id

u_id = user_df.user_id.sample(1).iloc[0]

user_df[user_df.user_id==u_id]

In [52]:
# pick city name from below sample cities

business_df.city.sample(10).iloc[:5]
# ['Brookline', 'Cambridge', 'Boston', 'Newton', 'Jamaica Plain',
#        'Somerville', 'Burlington', 'Quincy', 'Brighton', 'Allston']

In [69]:
# functional API recommender based on tf-idf vectorizer

def reco_resto(u_id,city):
    try:
        df = review_df[review_df.city==city].sort_values(['stars','useful','Year'],
                                                     ascending=[False,False,False]
                                                    )[['name','text']]

        df = df.groupby('name').head(10).reset_index(drop=True)
        df = df.groupby('name')['text'].apply(lambda x: ' '.join(x)).reset_index().sort_values('name')
        review_vec = tf.fit_transform(df.text.values).toarray()
        user_vec = get_top_user_reviews(u_id)
        distances = scipy.spatial.distance.cdist(user_vec,review_vec, 'cosine')
        distances = 1-distances
        df['cosine_distance'] = distances[0]
        return df.sort_values('cosine_distance',ascending=False)[['name','cosine_distance']].head(5)\
                                            .reset_index(drop=True)#.to_frame()
    except Exception as e:
        return 'Error-> '+str(e)    

In [72]:
reco_resto(u_id,'Boston')

In [59]:
!pip install -U sentence-transformers

In [60]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [66]:
# functional API recommender based on BERT model

def reco_resto_sbert(u_id,city):
    try:
        df = review_df[review_df.city==city].sort_values(['stars','useful','Year'],
                                                     ascending=[False,False,False]
                                                    )[['name','text']]

        df = df.groupby('name').head(10).reset_index(drop=True)
        df = df.groupby('name')['text'].apply(lambda x: ' '.join(x)).reset_index().sort_values('name')
        review_vec = model.encode(df.text.values)
        
        user_reviews =  review_df2[review_df2.user_id==u_id].sort_values(['stars','useful','Year'],
                                                 ascending=[False,False,False]
                                                )['text'].head(10).values
        
        user_vec = model.encode([' '.join(user_reviews)])
        print(review_vec.shape)
        print(user_vec.shape)
        distances = scipy.spatial.distance.cdist(user_vec,review_vec, 'cosine')
        distances = 1-distances
        df['cosine_distance'] = distances[0]
        return df.sort_values('cosine_distance',ascending=False)[['name','cosine_distance']].head(5)\
                                            .reset_index(drop=True)#.to_frame()
    except Exception as e:
        return 'Error-> '+str(e)    

In [71]:
reco_resto_sbert(u_id,'Boston')